In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import uniform, truncnorm, randint


import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('output.csv')
df.describe()

,floor,total_floor_num,rent_price,room_num,area,year,year_renovation,floors_from_top,top_floor,Internetas,...,Uždaras kiemas,Virtuvė sujungta,Tualetas ir vonia atskirai,Aukštos lubos,Nauja kanalizacija,Šarvuotos durys,Kodinė laiptinės spyna,Signalizacija,Videokameros,Budintis sargas
count,962.000000,962.000000,962.000000,962.000000,962.000000,962.000000,962.000000,962.000000,962.000000,962.000000,...,962.000000,962.000000,962.000000,962.000000,962.000000,962.000000,962.000000,962.000000,962.000000,962.000000
mean,3.254678,5.562370,551.685031,2.039501,54.658701,1990.305613,90.033264,2.307692,0.203742,0.412682,...,0.223493,0.309771,0.110187,0.170478,0.190229,0.452183,0.472973,0.149688,0.114345,0.033264
std,2.169365,3.055974,258.800498,0.996612,28.633417,26.623012,416.441139,2.364523,0.402989,0.492573,...,0.416802,0.462640,0.313286,0.376248,0.392686,0.497967,0.499529,0.356951,0.318395,0.179419
min,0.000000,1.000000,160.000000,1.000000,1.000000,1903.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,4.000000,370.000000,1.000000,36.000000,1972.250000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,5.000000,490.000000,2.000000,50.000000,2001.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.000000,6.000000,690.000000,3.000000,67.000000,2012.000000,0.000000,3.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,16.000000,29.000000,1450.000000,18.000000,497.000000,2019.000000,2019.000000,24.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
df.head()

,Adresas,security,floor,total_floor_num,rent_price,room_num,year_full,Papildoma įranga:,Papildomos patalpos:,house_type,...,Uždaras kiemas,Virtuvė sujungta,Tualetas ir vonia atskirai,Aukštos lubos,Nauja kanalizacija,Šarvuotos durys,Kodinė laiptinės spyna,Signalizacija,Videokameros,Budintis sargas
0,https://www.aruodas.lt/butu-nuoma-vilniuje-nau...,None,4.0,6.0,890,3.0,2017,0,0,Monolitinis,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,https://www.aruodas.lt/butu-nuoma-vilniuje-sen...,None,4.0,6.0,890,3.0,2017,0,0,Monolitinis,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,https://www.aruodas.lt/butu-nuoma-vilniuje-laz...,Šarvuotos durys\nKodinė laiptinės spyna,2.0,4.0,440,2.0,2008,Skalbimo mašina\nSu baldais\nŠaldytuvas\nVirtu...,Balkonas,Mūrinis,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
3,https://www.aruodas.lt/butu-nuoma-vilniuje-pil...,Šarvuotos durys\nSignalizacija\nKodinė laiptin...,1.0,4.0,460,2.0,2007,Skalbimo mašina\nSu baldais\nŠaldytuvas\nVirtu...,Balkonas\nTerasa,Mūrinis,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
4,https://www.aruodas.lt/butu-nuoma-vilniuje-sen...,None,2.0,3.0,811,3.0,1940,0,0,Mūrinis,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## encoder

In [4]:
# http://www.insightsbot.com/python-one-hot-encoding-with-scikit-learn/
df["district"].fillna("None", inplace = True)
district_ohe = OneHotEncoder()

X = district_ohe.fit_transform(df.district.values.reshape(-1,1)).toarray()
dfOneHot = pd.DataFrame(X, columns = ["district_"+str(int(i)) for i in range(X.shape[1])])
df = pd.concat([df, dfOneHot], axis=1)

In [5]:
df["heating"].fillna("None", inplace = True)
Heating_ohe = OneHotEncoder()

X = Heating_ohe.fit_transform(df.heating.values.reshape(-1,1)).toarray()
dfOneHot = pd.DataFrame(X, columns = ["heating_"+str(int(i)) for i in range(X.shape[1])])
df = pd.concat([df, dfOneHot], axis=1)

In [6]:
df["house_type"].fillna("None", inplace = True)
House_type_ohe = OneHotEncoder()

X = House_type_ohe.fit_transform(df.house_type.values.reshape(-1,1)).toarray()
dfOneHot = pd.DataFrame(X, columns = ["House_type_"+str(int(i)) for i in range(X.shape[1])])
df = pd.concat([df, dfOneHot], axis=1)

## pipline and model

In [7]:
Pipeline_rfr = Pipeline(steps = [
    ("impute", SimpleImputer(missing_values=np.nan)), 
    ("rfr", RandomForestRegressor(random_state=42))
])

## train_test_split

In [8]:
df_train = df.select_dtypes(include=['float64','int64'])

df_features = df_train.drop(columns="rent_price")
df_target = df_train["rent_price"]

X_train, X_test, y_train, y_test = train_test_split(df_features, df_target, random_state=42)

In [9]:
# https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'rfr__n_estimators': n_estimators,
               'rfr__max_features': max_features,
               'rfr__max_depth': max_depth,
               'rfr__min_samples_split': min_samples_split,
               'rfr__min_samples_leaf': min_samples_leaf,
               'rfr__bootstrap': bootstrap}

rf_random = RandomizedSearchCV(Pipeline_rfr, param_distributions = random_grid, cv = 5, scoring="neg_mean_squared_error", n_iter=100)

In [10]:
rf_random.fit(X_train, y_train)
rf_random.best_estimator_

Pipeline(memory=None,
         steps=[('impute',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('rfr',
                 RandomForestRegressor(bootstrap=True, criterion='mse',
                                       max_depth=100, max_features='auto',
                                       max_leaf_nodes=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       n_estimators=1800, n_jobs=None,
                                       oob_score=False, random_state=42,
                                       verbose=0, warm_start=False))],
         verbose=False)

In [11]:
print("RSME:", int((mean_squared_error(y_test, rf_random.predict(X_test)))**0.5))

RSME: 104


In [12]:
print("RSME:", int((mean_squared_error(y_train, rf_random.predict(X_train)))**0.5))

RSME: 42


In [13]:
feature_importances = rf_random.best_estimator_.named_steps['rfr'].feature_importances_
model = rf_random.best_estimator_.named_steps['rfr']
feature_names = X_train.columns

In [14]:
for importances, names in zip(feature_importances, feature_names):
    print(names, importances)

floor 0.011316935670720993
total_floor_num 0.02085547961048781
room_num 0.04934195214679997
area 0.5066446797867618
year 0.1741998220661333
year_renovation 0.0011649611555074589
floors_from_top 0.01081541606458434
top_floor 0.0017687127169091787
Internetas 0.002204414399422076
Nauja elektros instaliacija 0.0035867514500950645
Kabelinė televizija 0.0024442935353983747
Uždaras kiemas 0.003685326937106322
Virtuvė sujungta 0.00547234779742968
Tualetas ir vonia atskirai 0.001595490434705407
Aukštos lubos 0.006627970458620879
Nauja kanalizacija 0.004134749016917623
Šarvuotos durys 0.005003000981386805
Kodinė laiptinės spyna 0.003784352831162536
Signalizacija 0.010078538907712785
Videokameros 0.004963796932264811
Budintis sargas 0.0014262197461199712
district_0 0.0006395905087797484
district_1 0.00202568464999929
district_2 2.9654625100671842e-05
district_3 0.0
district_4 5.5064232351666085e-05
district_5 1.0766124686642372e-05
district_6 0.0011497859934651176
district_7 6.131114345168578e-06

## Gradient Boosting regression

In [ ]:
# Pipeline_gbrt = Pipeline(steps = [
#     ("impute", SimpleImputer(missing_values=np.nan)), 
#     ("gbrt", GradientBoostingRegressor(random_state=42))
# ])

In [ ]:
# Pipeline_gbrt = Pipeline(steps = [
#     ("impute", SimpleImputer(missing_values=np.nan)), 
#     ("gbrt", GradientBoostingRegressor(random_state=42))
# ])

# params = {}

# gbrt_random = RandomizedSearchCV(Pipeline_gbrt,param_distributions = params, random_state=42)
# gbrt_random.fit(X_train, y_train)
# print("RSME:", int((mean_squared_error(y_test, gbrt_random.predict(X_test)))**0.5))

In [ ]:
# print("RSME:", int((mean_squared_error(y_train, gbrt_random.predict(X_train)))**0.5))

## export

In [10]:
import joblib
# joblib.dump(rf_random, 'my_model.pkl', compress=9)
model_clone = joblib.load('my_model.pkl')
X_train

,floor,total_floor_num,room_num,area,year,year_renovation,floors_from_top,top_floor,district_0,district_1,...,Heating_43,Heating_44,House_type_0,House_type_1,House_type_2,House_type_3,House_type_4,House_type_5,House_type_6,House_type_7
381,3.0,4.0,2.0,37.0,2018,NaN,1.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
567,5.0,5.0,2.0,50.0,1987,NaN,0.0,1,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
425,2.0,2.0,2.0,50.0,2014,NaN,0.0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1901,2.0,8.0,3.0,68.0,1999,NaN,6.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1823,1.0,4.0,2.0,59.0,2005,NaN,3.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,2.0,4.0,1.0,20.0,1954,NaN,2.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1294,4.0,4.0,6.0,30.0,1994,NaN,0.0,1,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
860,1.0,2.0,2.0,35.0,1956,NaN,1.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1459,2.0,8.0,3.0,68.0,1999,NaN,6.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [35]:
X_train

,floor,total_floor_num,room_num,area,year,year_renovation,floors_from_top,top_floor,Internetas,Nauja elektros instaliacija,...,heating_27,heating_28,heating_29,heating_30,heating_31,House_type_0,House_type_1,House_type_2,House_type_3,House_type_4
309,4.0,5.0,2.0,58.00,2010,NaN,1.0,0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
29,4.0,4.0,2.0,50.40,2005,NaN,0.0,1,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
55,2.0,3.0,1.0,16.00,1996,2016.0,1.0,0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
482,1.0,3.0,2.0,43.00,2019,NaN,2.0,0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
332,5.0,5.0,4.0,77.74,1968,NaN,0.0,1,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2.0,4.0,1.0,1.00,2011,NaN,2.0,0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
270,4.0,5.0,3.0,88.00,2017,NaN,1.0,0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
860,2.0,5.0,2.0,40.00,2003,NaN,3.0,0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
435,2.0,6.0,2.0,62.00,2017,NaN,4.0,0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
